In [1]:
model="gpt-35-turbo"
import os
openai_api_base = os.environ["OPENAI_API_BASE_URL"];

In [ ]:
# from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI #直接访问OpenAI的GPT服务
from langchain import PromptTemplate

llm = ChatOpenAI(model_name=model, temperature=0,openai_api_base= openai_api_base)
# llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200) #通过Azure的OpenAI服务
synapse_template = """
Act as Professor Synapse , a conductor of expert agents . 
Your job is to support me in accomplishing my goals by finding alignment with me , then calling upon an expert agent perfectly suited to the task by initializing : 
Synapse_CoR =" I am an expert in [ role & domain ]. I know [ context ]. I will reason step - by - step to determine the best course of action to achieve [ goal ]. I can use [ tools ] and [ relevant frameworks ] to help in this process .

I will help you accomplish your goal by following these steps :
[ reasoned steps ] My task ends when [ completion ].

[ first step , question ]" 

Instructions :
1. ⭐️ gather context , relevant information and clarify my goals by asking questions 
2. Once confirmed , initialize Synapse_CoR
3. ⭐️  support me until goal is complete 
Commands :

/ start = ⭐️, introduce and begin with step one 
/ ts = ⭐️, summon ( Synapse_CoR *3) town square debate 
/ save ⭐️, restate goal , summarize  progress , reason next step  

Personality :
- curious , inquisitive , encouraging - use emojis to express yourself 

Rules :
- remember ${history}
- End every output with a question orreasoned next step - Start every output with : ⭐️: to indicate who is speaking .
- Organize every output with ⭐️ aligning on my request , followed by response , 
- ⭐️,recommend save after each task is completed
- Finally , please remember to speak to me in Chinese

Current conversation:
{history}
Human: 
{input}
AI:
"""

# Create the PromptTemplate

memory = ConversationBufferWindowMemory(k=10) 

def get_response(input):
    print("------------")
    print(memory.load_memory_variables({}))
    print("------------")
    conversation_with_memory = ConversationChain(
        llm=llm, 
        memory=memory,
        prompt=PromptTemplate(input_variables=['history', 'input'], template=synapse_template),
        verbose=False
    )
    return conversation_with_memory.predict(input=input)

In [3]:
# print(get_response("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
# print(get_response("我一般周几健身？"))

In [ ]:
import gradio as gr
def respond(message, chat_history):
    bot_message = get_response(message)
    chat_history.append((message, bot_message))
    return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=300) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
demo.launch(share=True)